In [1]:
import os
# disable 
import glob
import shutil

# import load_model
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib
matplotlib.use("Qt5Agg")

from enum import Enum
from tqdm import tqdm


In [10]:
log_path = '2023-01-30_145630'
model_iteration = '80'

initial_balance = 100
balance = initial_balance
coin_amount = 0
risk = 0.2

In [11]:
prefix = './Results'
model_path = os.path.join(prefix, log_path, 'models', f'model_{model_iteration}.h5')
history_path = os.path.join(prefix, log_path, 'reward_history.csv')
image_path = os.path.join(prefix, log_path, 'images')

### Load functions

In [4]:
original_items = ['event_time','original_open', 'original_close', 'original_high', 'original_low', 'original_volume']

class ActionSpace(Enum):
    BUY = 0
    HOLD = 1
    SELL = 2

def plot_data(data, ax):
    if ax is None:
        ax = plt.gca()
    ax.plot(data)
    # add trend line
    ax.plot(np.poly1d(np.polyfit(range(len(data)), data, 2))(range(len(data))))
    ax.set_xlabel("Episode")
    ax.legend(["Reward"])
    ax.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False,
                     bottom=True, top=False, left=True, right=True)
    ax.grid(axis='y')
    reward_save_path = os.path.join(image_path, f'average_reward_{model_iteration}.png')

def sequence_generator(data, batch_size, max_steps, start_index=0):

        
    for i in range(start_index + batch_size + 1, max_steps + start_index):
        yield data[i - batch_size:i]

def get_train_env_data(data):
    item = pd.DataFrame(data)
    return item[original_items], item.drop(original_items, axis=1)

def plot_buy_sell(data, ax):
    if ax is None:
        ax = plt.gca()

    

    width = .4
    width2 = .04

    #define colors to use2023-01-18_20:43:23
    col1 = 'green'
    col2 = 'red'

    buys = 0
    sells = 0
    holds = 0

    for i in range(0, len(data)):
        item = data[i]
        item_candlestick = item[0]
        action = item[1]
        index = item[2]



        if item_candlestick['original_close'] >= item_candlestick['original_open']:
            height = item_candlestick['original_close'] - item_candlestick['original_open']
            height_max = item_candlestick['original_high'] - item_candlestick['original_low']
            ax.bar(index, height, width, item_candlestick['original_open'], color=col1)
            ax.bar(index, height_max, width2, item_candlestick['original_low'], color=col1)
        else:
            height = item_candlestick['original_open'] - item_candlestick['original_close']
            height_max = item_candlestick['original_high'] - item_candlestick['original_low']
            ax.bar(index, height, width,  item_candlestick['original_close'], color=col2)
            ax.bar(index, height_max, width2,  item_candlestick['original_low'], color=col2)

        if action == ActionSpace.BUY:
            buys += 1
            ax.scatter(index, item_candlestick['original_close'], marker='o', color='cyan', s=14)

        elif action == ActionSpace.SELL:
            sells += 1
            ax.scatter(index, item_candlestick['original_close'], marker='o', color='magenta', s=14)

        elif action == ActionSpace.HOLD:
            holds += 1
            ax.scatter(index, item_candlestick['original_close'], marker='o', color='black', s=14)

    # set y ticks to both sides
    


    ax.set_ylabel("Price")
    ax.set_xlabel("Minutes")
    ax.legend()

    ax.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False,
                     bottom=True, top=False, left=True, right=True)
    ax.grid(axis='y')
    return buys, sells, holds

def create_count_plot(buys, sells, holds, ax):
    # create count plot with buys, sells, holds

    ax.bar(0, buys, width=.4, color='cyan', label=f'Buy: {buys}')
    ax.bar(1, sells, width=.4, color='magenta', label=f'Sell: {sells}')
    ax.bar(2, holds, width=.4, color='black', label=f'Hold: {holds}')

    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(['Buy', 'Sell', 'Hold'])

    # show values on bars
    
    ax.set_ylabel("Count")
    ax.set_xlabel("Action")
    ax.legend()

    ax.grid(axis='y')



def buy(current_price):
    global balance, risk, coin_amount

    percentage = (balance * risk) / current_price

    balance -= percentage * current_price
    coin_amount += percentage



        
def sell(current_price):

    global balance, coin_amount

    balance += coin_amount * current_price
    coin_amount = 0


### Predict sequences

In [5]:
# load AI model from Models


# read data from file into pandas dataframe
df = pd.read_csv("./Data/dataset/VETUSDT.csv")



# loop through all sequences and predict buy hold or sell and put the results on a graph


LOOKBACK_WINDOW = 100
ITEMS = 200


# will loop all training data and predict using the last saved model of that training session and save it to a graph image.
for root, dirs, files in os.walk("./Results/"):
    timestamps_progres = tqdm(dirs)
    for dir in timestamps_progres:
        timestamp = dir
        for models_root, models_dirs, models_files in os.walk(os.path.join(root, dir, 'models')):
            models_files.sort(key=lambda x: os.path.getmtime(os.path.join(root, dir, 'models', x)))
            if len(models_files) == 0 or len(models_files) < 4:
                # remove the training attempt if it has no models
                shutil.rmtree(os.path.join(root, dir))
                continue
        
            file = models_files[-1]

            # get index of model in models_files
            model_index = models_files.index(file)
            timestamps_progres.set_description(f"Timestamp: {timestamp}, Model: {models_files.index(file)}, Models: {len(models_files)}")
            model_path = os.path.join(root, dir, 'models', file)
            try:
                model = load_model(model_path, compile=False)
                model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
                training_history = pd.read_csv(os.path.join(root, dir, 'reward_history.csv'))
                buy_sells = []
                seq_gen = sequence_generator(df, LOOKBACK_WINDOW, ITEMS, 20000)
                for i in range(0, ITEMS - LOOKBACK_WINDOW):
                    try:
                        item = next(seq_gen)
                    except:
                        break
                    item = item.copy()
                    item['balance'] = balance
                    item['coin_amount'] = coin_amount
                    train, env = get_train_env_data(item)
                    env = env.to_numpy()
                    prediction = model.predict(env.reshape(1, LOOKBACK_WINDOW, 19), verbose=0)
                    action = ActionSpace(np.argmax(prediction))

                    if action == ActionSpace.BUY:
                        buy(item.iloc[-1]['original_close'])

                    elif action == ActionSpace.SELL:
                        sell(item.iloc[-1]['original_close'])
                        
                    buy_sells.append((train.iloc[-1], action, i))

                balance = initial_balance
                coin_amount = 0

                fig = plt.figure(figsize=(20, 10))
                grid = plt.GridSpec(2, 2)
                fig.tight_layout(pad=3)

                ax_1 = fig.add_subplot(grid[0, 0])
                ax_2 = fig.add_subplot(grid[1, :2])
                ax_3 = fig.add_subplot(grid[0, 1])
                plot_data(training_history['reward'], ax_1)
                buys, sells, holds = plot_buy_sell(buy_sells, ax_2)
                create_count_plot(buys, sells, holds, ax_3)


                prediction_path = os.path.join(root, dir, 'images', f'prediction_{file}.png')
                plt.savefig(prediction_path)
                matplotlib.pyplot.close()
            except Exception as ex:
                print(ex)
                continue



Timestamp: 2023-01-25_191813, Model: 13, Models: 14:   0%|          | 0/122 [00:00<?, ?it/s]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
Timestamp: 2023-01-26_090933, Model: 6, Models: 7:   1%|          | 1/122 [00:11<22:46, 11.30s/it]  No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
Timestamp: 2023-01-26_120007, Model: 5, Models: 6:   2%|▏         | 2/122 [00:20<20:09, 10.08s/it]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
Timestamp: 2023-01-26_140618, Model: 5, Models: 6:   2%|▏         | 3/122 [00:29<19:16,  9.72s/it]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called wi

KeyboardInterrupt: 

In [13]:
# load AI model from Models


# read data from file into pandas dataframe
df = pd.read_csv("./Data/dataset/VETUSDT.csv")



# loop through all sequences and predict buy hold or sell and put the results on a graph


LOOKBACK_WINDOW = 100
ITEMS = 200



model = load_model(model_path, compile=False)
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
training_history = pd.read_csv(history_path)
buy_sells = []
seq_gen = sequence_generator(df, LOOKBACK_WINDOW, ITEMS, 20000)
for i in range(0, ITEMS - LOOKBACK_WINDOW):
    try:
        item = next(seq_gen)
    except:
        break
    item = item.copy()
    item['balance'] = balance
    item['coin_amount'] = coin_amount
    train, env = get_train_env_data(item)
    env = env.to_numpy()
    prediction = model.predict(env.reshape(1, LOOKBACK_WINDOW, 19), verbose=0)
    action = ActionSpace(np.argmax(prediction))

    if action == ActionSpace.BUY:
        buy(item.iloc[-1]['original_close'])

    elif action == ActionSpace.SELL:
        sell(item.iloc[-1]['original_close'])
        
    buy_sells.append((train.iloc[-1], action, i))

balance = initial_balance
coin_amount = 0

### Show graphs

In [15]:
fig = plt.figure(figsize=(20, 10))
grid = plt.GridSpec(2, 2)
fig.tight_layout(pad=3)

ax_1 = fig.add_subplot(grid[0, 0])
ax_2 = fig.add_subplot(grid[1, :2])
ax_3 = fig.add_subplot(grid[0, 1])
plot_data(training_history['reward'], ax_1)
buys, sells, holds = plot_buy_sell(buy_sells, ax_2)
create_count_plot(buys, sells, holds, ax_3)


prediction_path = os.path.join(image_path, f'prediction_{model_iteration}.png')
plt.savefig(prediction_path)
plt.show(block=False)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
